# Дизайн стабильности

## 0. Постановка задачи



*весь код - для запуска в колабе. перед запуском включите gpu*

Возьмем структуру бактериальной бета-глюкозидазы B и попробуем улучшить ее стабильность разными методами

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time


In [ ]:
! wget https://files.rcsb.org/download/2JIE.pdb # скачаем файл со структурой

Визуализируем структуру


In [ ]:
!pip install py3Dmol
import py3Dmol

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/2JIE.pdb','r').read(),'pdb') # добавить модель из файла

p.setStyle({'cartoon': {'color':'green'}}) # отобразить все в виде cartoon и покрасить в зеленый

selection = {'resn':'G2F','byres':'true', 'expand': 5} # выбрать остатки на расстоянии не дальше 5 ангстрем от лиганда
p.setStyle(selection,{'stick':{'colorscheme':'greenCarbon'}}) # выбранное отобразить в виде палочек

p.setStyle({'resn': 'G2F'},{'sphere': {'colorscheme': 'greenCarbon'}}) # лиганд отобразить в виде сфер

p.zoomTo() # центрировать структуру

p.show()

Белок состоит из одной белковой цепи и связывает 2-F-глюкозу. Предположим, что остатки, близкие к лиганду, участвуют в его связывании и катализе, а потому их мутировать не будем, чтобы не потерять функцию.

Выберем и запомним эти остатки

In [ ]:
!pip install Bio
from Bio import PDB
from Bio.PDB.PDBParser import PDBParser

In [ ]:
parser=PDBParser()
structure=parser.get_structure("2JIE",'2JIE.pdb')

In [ ]:
# проитерируем по всем атомам структуры и запишем атомы лиганда в список
lig_atoms=[]
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_resname()=='G2F':
                lig_atoms.extend(residue.get_atoms())
lig_atoms

In [ ]:
# найдем остатки, участвующие в связывании лиганда

lresi=[]
for model in structure:

    ...

lresi

## 1. FoldX

from https://github.com/leandroradusky/pyfoldx/blob/master/notebooks/StructureUsage.ipynb

FoldX - популярный инструмент, используемый для дизайна белков и оценки энергий.

In [ ]:
! pip install pyfoldx

#! wget https://foldxsuite.crg.eu/system/files/foldx5Linux64.tar_.gz # для скачивания надо зарегистрироваться
! gdown 152JYCr7Wmmp092aXFZxPGwMQegblWAlm # ссылка на заранее скачанный файл

! mkdir foldx
! tar -xvzf foldx5Linux64.tar_.gz -C foldx -v

In [ ]:
import os
os.environ['FOLDX_BINARY'] = '/content/foldx/foldx_20231231'

from pyfoldx.structure import Structure

Загрузим структуру

In [ ]:
st=Structure("2JIE",'2JIE.pdb')

Рассчитаем энергию deltaG

In [ ]:
st.getTotalEnergy()

Рассчитаем энергию для каждого остатка

In [ ]:
#! $FOLDX_BINARY --command=SequenceDetail --pdb=2JIE.pdb  # можно также запускать из консоли

res_df=st.getResiduesEnergy()
res_df['total']=res_df['total'].apply(float)
res_df

У программы FoldX есть метод repair, который минимизирует энергию структуры, подбирая оптимальные с точки зрения энергии ротамеры.

In [ ]:
%%time
stRepaired=st.repair()

In [ ]:
print( "Original Structure energy" )
print( float(st.getTotalEnergy().total) )
print()
print( "Repaired Structure energy" )
print( float(stRepaired.getTotalEnergy().total) )


Визуализируем оптимизацию

In [ ]:
# создадим новый датасет с энергиями оригинальной и оптимизированной миодели
combinedDf = res_df[["sec_struct","total" ]]
combinedDf['repaired'] = stRepaired.getResiduesEnergy(consider_waters=True)[["total"]].astype(float)
combinedDf.columns=["Secondary structure",'Original', "Repaired"]

fig, (ax1, ax2) = plt.subplots(figsize=(20, 3), nrows=2)

# изобразим энергии в виде heatmap
sns.heatmap(combinedDf[['Original', "Repaired"]].T.astype('float64'), ax=ax1)
ax1.set_xticks([])
ax1.set_xlabel('')

# отобразим элементы вторичной структуры в виде heatmap
value_to_int = {value: i for i, value in enumerate([ 'a', '3', 'B', 'b', 'E', 'T','*', 'n', 'c'])}
sns.heatmap(combinedDf[["Secondary structure"]].T.replace(value_to_int),
            ax=ax2, cmap=sns.color_palette("Spectral", len(value_to_int)) )
colorbar = ax2.collections[0].colorbar
r = colorbar.vmax - colorbar.vmin
colorbar.set_ticks([(len(value_to_int)-1)/len(value_to_int)*(0.5 + i) for i in range(len(value_to_int))])
colorbar.set_ticklabels([ 'a', '3', 'B', 'b', 'E', 'T','*', 'n', 'c'])

plt.plot()

Выберем самые энергетически неэффективные остатки и попробуем их мутировать, чтобы улучшить общую стабильность белка

In [ ]:
combinedDf.Repaired.hist()

In [ ]:
pos_to_mutate=combinedDf.Repaired.sort_values(ascending=False).index # отсортируем остатки по энергиям
pos_to_mutate=[int(x[2]) for x in pos_to_mutate]


Выберем самые энергетически неэффективные остатки для мутации. Убедимся, что остатки, подвергающиеся мутации, не участвуют в связывании лиганда

In [ ]:
...

Оценим изменение энергии при заменах каждого остатка из выбранных

In [ ]:
%%time
mut_df=stRepaired.positionScan(positions=pos_to_mutate, chain='A') # занимает минут 5 на 1 мутацию
mut_df

Выберем мутации и промутируем структуру

In [ ]:
mut_df.values.argmin(axis=0) # выбираем для каждой позиции наиболее энегретически выгодные замены

In [ ]:
mutations=[x[0][0]+'A'+x[0][1:]+x[1] for x in zip(mut_df.columns,mut_df.index[mut_df.values.argmin(axis=0)])] # переформатируем мутации в нужный формат (resname+chain+resid+mut)
mutations

In [ ]:
%%time
mut_df, mut_ens, wt_ens = stRepaired.mutate(','.join(mutations)+';',number_of_runs=5)
mut_df

Оценим выигрыш по энергии

In [ ]:
print( "Original Structure energy" )
print( wt_ens.getTotalEnergy().total.apply(float).mean())
print()
print( "Mutated Structure energy" )
print( mut_ens.getTotalEnergy().total.apply(float).mean())


Визуализируем мутантный белок

In [ ]:
with open('/content/2JIE_foldx.pdb','w') as f: # запишем мутантный белок в файл
    f.write('\n'.join(mut_ens.frames[3].toPdb()))

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/2JIE_foldx.pdb','r').read(),'pdb')
p.setStyle({'cartoon': {'color':'green'}})
p.setStyle({'resi': pos_to_mutate},{'stick':{'colorscheme':'blueCarbon'}})

p.zoomTo()
p.show()

## 2. trDesign

trDesign - набор методов для дизайна белков, использующий trRosetta - нейросеть для предсказания структуры белка

from https://github.com/gjoni/trDesign/blob/master/02-GD/notebooks/TrDesign_GD_demo.ipynb

![image](https://camo.githubusercontent.com/eaafe9b0da4f6867a9503ced271af659d15f1c7d435bc7209f87f88ed4f55908/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f676a6f6e692f747244657369676e2f6d61737465722f30322d47442f6e6f7465626f6f6b732f6669677572655f312e706e67)

Один из вариантов его применения - дизайн последовательности под известный фолд. Для этого на вход модели подается случайная последовательность. Разница между предсказанной и целевой структурой используется для расчета градиента, в котором последовательность оптимизируется методом стохастического градиентного спуска. Было показано, что данный алгоритм предпочитает выбирать наиболее энергетически выгодные последовательности.

In [ ]:
!wget -qnc https://raw.githubusercontent.com/gjoni/trDesign/master/02-GD/models.py
!wget -qnc https://raw.githubusercontent.com/gjoni/trDesign/master/02-GD/utils.py
!wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/design/to_pdb.py
!wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/models.zip
!wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/bkgr_models.zip
!unzip -qqo models.zip
!unzip -qqo bkgr_models.zip

In [ ]:
from utils import *
from models import *
from to_pdb import *

Загрузим белок и получим признаки, которые будут использоваться в модели. Это закодированная последовательность белка, и попарные расстояния и углы между остатками.

In [ ]:
pdb = prep_input("/content/2JIE.pdb", chain="A", mask_gaps=False)
_feat = pdb["feat"]
_seq = np.eye(20)[AA_to_N(pdb["seq"])]

In [ ]:
plt.figure(figsize=(20,2) )
plt.imshow(_seq.T, cmap="binary")

In [ ]:
plt.figure(figsize=(20,4) )

for n,(k,v) in enumerate(split_feat(_feat).items()):
    plt.subplot(1,4,n+1); plt.title(k)
    plt.imshow(np.squeeze(v).argmax(-1),cmap="binary")
plt.show()

![image2](https://camo.githubusercontent.com/cdd8cf473fe975cb15db2b5c2789bd622a0ca11b3994726976ba0d5fbccede7e/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f676a6f6e692f747244657369676e2f6d61737465722f30322d47442f6e6f7465626f6f6b732f36442e706e67)

Загрузим модель и запустим дизайн

In [ ]:
# вопрос: как ограничить модель, чтобы она оптимизировала только определенную часть последовательности?
# https://github.com/gjoni/trDesign/issues/2

...

for i in range(_seq.shape[1]):
    if (i+5) in lresi: # обращаем внимание на несовпадение нумераций остатков
        _seq[:,i,:]=_seq[:,i,:]*1e8 # остаткам, которые надо сохранить, присвоим большие значения, чтобы они не менялись при шаге оптимизации

In [ ]:
model = mk_design_model(add_pdb=True, n_models=1)

In [ ]:
%%time
design = model.design(inputs={"I":_seq[None],"pdb":_feat[None]}, opt_iter=200, opt_method="ADAM", num=1)

Визуализируем полученную последовательность

In [ ]:
plt.figure(figsize=(20,2) )
plt.subplot(211)
plt.imshow(np.clip(_seq.T, 0, 1), cmap="binary")
plt.subplot(212)
plt.imshow(np.clip(design['I'].T, 0, 1), cmap="binary")
plt.plot()

In [ ]:
print(pdb["seq"])
print(N_to_AA(design["I"].argmax(-1)))

In [ ]:
plt.figure(figsize=(20,8) )

for n,(k,v) in enumerate(split_feat(_feat).items()):
    plt.subplot(2,4,n+1); plt.title(k)
    plt.imshow(np.squeeze(v).argmax(-1),cmap="binary")

for n,(k,v) in enumerate(split_feat(design["feat"]).items()):
    plt.subplot(2,4,4+n+1); plt.title(k)
    plt.imshow(np.squeeze(v).argmax(-1),cmap="binary")
plt.show()

Сохраним в файл

In [ ]:
seq = N_to_AA(np.squeeze(design["I"]).argmax(-1))[0]
xyz, dm = feat_to_xyz(np.squeeze(design["feat"]))
save_PDB("/content/2JIE_trdesign.pdb", xyz, dm, seq)

Визуализируем

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open('/content/2JIE_trdesign.pdb','r').read(),'pdb')
p.setStyle({'cartoon': {'colorscheme':'greenCarbon'}})
#p.setStyle({'sticks': {'colorscheme':'greenCarbon'}})
p.zoomTo()
p.show()

Боковых цепей нам не завезли, попробуем реконструировать их с помощью foldX

In [ ]:
%%time
! $FOLDX_BINARY --command=Optimize --pdb=2JIE_trdesign.pdb >> foldx.log

Рассчитаем энергии сгенерированной структуры

In [ ]:
st_tr=Structure("2JIE_trdesign",'/content/Optimized_2JIE_trdesign.pdb')
st_tr.getTotalEnergy()

Попробуем оптимизировать структуру

In [ ]:

st_trRepaired=st_tr.repair()
print( "Original Structure energy" )
print( float(st_tr.getTotalEnergy().total) )
print()
print( "Repaired Structure energy" )
print( float(st_trRepaired.getTotalEnergy().total) )


Попробуем получить структуру сгенерированной нами последовательности не с помощью trRosetta, а другим методом, например, гомологичным моделированием (смотри предыдущий прак)

In [ ]:
! cd /tmp/ ; wget https://salilab.org/modeller/10.1/modeller_10.1-1_amd64.deb
! env KEY_MODELLER="MODELIRANJE" dpkg -i /tmp/modeller_10.1-1_amd64.deb
! echo -e "install_dir = r'/usr/lib/modeller10.1'\nlicense = 'MODELIRANJE'" > /usr/lib/modeller10.1/modlib/modeller/config.py

import sys
sys.path.append('/usr/lib/python3.9/dist-packages')


In [ ]:
import modeller
from modeller import automodel

env=modeller.environ()
aln=modeller.alignment(env)


aln.append_sequence(N_to_AA(design["I"].argmax(-1))[0])
aln[0].code='mut'
aln

pdbl = PDB.PDBList()
mdl = modeller.model(env, file='2JIE.pdb')
aln.append_model(mdl, atom_files='2JIE.pdb', align_codes='2JIE')
aln.salign()
aln.write(file='ali.ali', alignment_format='PIR')

a = automodel.automodel(env, alnfile='ali.ali', knowns= '2JIE', sequence = 'mut' )

a.starting_model = 1
a.ending_model = 2
a.make()

In [ ]:
...

## 3. Домашняя работа

Используйте RaSP (https://github.com/KULL-Centre/_2022_ML-ddG-Blaabjerg/) для предсказания эффектов мутаций. Выберите несколько наиболее эффективных по мнению RaSP мутаций, внесите их в структуру и оцените изменения энергии